In [1]:
import os

from pylatex import (
	Document,
	Section,
	Subsection,
	Command,
	Package,
)
from pylatex.utils import (
	italic,
	NoEscape,
)
from pylatex.section import (
	Paragraph,
)

section
	subsection
	

In [2]:
def get_doc(corpus):
	geometry_options = {"margin": "2cm"}
	doc = Document(geometry_options=geometry_options)
	doc.packages.append(Package("expex"))

	doc.append(Command("title", NoEscape(r"GlossySnake: A LaTeX Application \\ \large on Die Leiden des jungen Werther")))
	doc.append(Command("author", "Chanjin Park a.k.a. \"Chamchi\" (parkchamchi@gmail.com)"))
	doc.append(Command("maketitle"))

	with doc.create(Section("On GlossySnake")):
		introduction = """
			This is a \LaTeX{}	 Proof-of-Concept of the GlossySnake project.
			The GlossySnake project aims to approximate the Interlinear Translation corpora via machine-glossing.
			By the nature of the machine-glossing, the inaccurate and malformatted glossing is expected.
			Visit https://github.com/parkchamchi/GlossySnake for more information.
		"""

		doc.append(NoEscape(introduction.replace('\n', '').replace('\t', ' ').strip()))

	#doc.append(Command("lingexbreakpenalty=10000"))

	corpus_title = "Die Leiden des jungen Werther (1774)"
	with doc.create(Section(corpus_title)):
		for p in corpus["paragraphs"]:
			if p["is_delimiter"]:
				continue

			#with doc.create(Paragraph("")):
			txt_s = []
			gloss_s = []

			#Ignoring delimters.
			for t in p["tokens"]:
				if t["is_delimiter"]:
					if '\n' in t["txt"]:
						txt_s.append('+')
						#gloss_s.append('+')
					continue
				
				txt_s.append(t["txt"])

				gloss = t["gloss"]
				if gloss == "!UNKNOWN":
					gloss = "-"
				gloss_s.append(gloss)

			if [e for e in txt_s if e.strip()] == []:
				continue

			#expex
			#with doc.create(Subsection("")):
			doc.append(Command("exdisplay"))
			"""
			doc.append(Command("begingl", None, {
				"everygla": '',
				"everyglb": r"\tt",
				"glwordalign": "center",
			}))
			"""
			doc.append(NoEscape(
				r"\begingl["
				r"everygla=\LARGE,everyglb=\tt,"
				r"glwordalign=center,extraglskip=0.75em,aboveglbskip=0.25em,"
				r"glbreaking,glspace=0.75em"
				r"]"
			))

			doc.append(Command("gla " + " ".join(['{'+e+'}' for e in txt_s]) + " // "))
			doc.append(Command("glb " + " ".join(['{'+e+'}' for e in gloss_s]) + " // "))
			#doc.append(Command("glft // "))

			doc.append(Command("endgl"))
			doc.append(Command("xe"))
			
	return doc


In [3]:
import json

corpus = "./Die Leiden des jungen Werther.corpus.json"
with open(corpus, "rt", encoding="utf-8") as fin:
	corpus = json.load(fin)

In [4]:
doc = get_doc(corpus)
doc.generate_pdf("full", clean_tex=False)